In [346]:
import numpy as np
data_path = "/Users/haruhi/Desktop/AMATH 482/hw1/subdata.npy"
d = np.load(data_path) # huge matrix of size 262144 x 49 (columns contain flattened 3d matrix of size 64x64x64)

In [347]:
# import libraries for plotting isosurfaces
import plotly
import plotly.graph_objs as go
# utility for clearing output of cell as loop runs in notebook
from IPython.display import clear_output
%matplotlib notebook
import matplotlib.pyplot as plt
import pywt
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
# plot the data in time

L = 10 # length of spatial domain (cube of side L = 2*10)
N_grid = 64 # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # frequency grid for one coordinate
# K_grid = np.fft.fftshift(K_grid)

xv, yv, zv = np.meshgrid(x, y, z) # generate 3D meshgrid for plotting
kx, ky, kz = np.meshgrid(K_grid, K_grid, K_grid)

In [348]:
# averaging the Fourier Transform
# for each time step, use Fourier Transform for the 3-D matrix
# and calculate the average fft over time

# reconstruct each of the 49 cubes
sum = np.zeros((N_grid, N_grid, N_grid))
for t in range(49):
    signal = np.reshape(d[:, t], (N_grid, N_grid, N_grid))
    sum = sum + np.fft.fftshift(np.fft.fftn(signal))

fft_ave = abs(sum/49)

# determine the center frequency
max_indices = np.unravel_index(fft_ave.argmax(), fft_ave.shape)
# max_indices = (26, 17, 55)
center = [kx[max_indices], ky[max_indices], kz[max_indices]]
print(max_indices)
print(center)

(39, 49, 10)
[5.340707511102648, 2.199114857512855, -6.911503837897545]


In [349]:
# 3D Gaussian filter

def g(x, y, z, s):
    val = np.exp(-((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)/(2*s**2))
    return val

sigma = 2
g_vals = g(xv, yv, zv, sigma)

fig, ax = plt.subplots(figsize=(6,6))
z0 = 32
ax.contourf(kx[:,:,z0], ky[:,:,z0], g_vals[:,:,z0])
ax.set_title('Gaussian filter')
ax.set_xlabel('kx')
ax.set_ylabel('ky')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'ky')

In [350]:
# create another fft
center = np.ndarray((3, 49))
for t in range(49):
    # fft
    signal = np.reshape(d[:, t], (N_grid, N_grid, N_grid))
    signal_fft = np.fft.fftshift(np.fft.fftn(signal))
    # apply filter
    signal_fft_filtered = g_vals * signal_fft
    # inverse fft
    signal_filtered = np.fft.ifftn(np.fft.ifftshift(signal_fft_filtered))
    signal_filtered = abs(signal_filtered)
    max_indices = np.unravel_index(signal_filtered.argmax(), signal_filtered.shape)

    center[:,t] = (xv[max_indices], yv[max_indices], zv[max_indices])

plt.figure(2)
ax = plt.axes(projection='3d')
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.set_zlabel('z',fontsize=15)
ax.set_title('Path of submarine',fontsize=20)
ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])
ax.set_zlim([-10, 10])
ax.plot3D(center[0,:], center[1,:], center[2,:])
xy = center[0:2,:]
df = pd.DataFrame(xy).T
df.to_excel(excel_writer = "/Users/haruhi/Desktop/AMATH 482/hw1/coordinate.xlsx")

<IPython.core.display.Javascript object>

In [351]:
plt.figure(3)
plt.plot(center[0,:], center[1,:])
plt.grid(color='g', linestyle='--', linewidth=0.5)
plt.xlim([-10,10])
plt.ylim([-10,10])
plt.xlabel('x', fontsize=15)
plt.ylabel('y', fontsize=15)
plt.title('x-y Coordinate of the Submarine', fontsize=15)
plt.show()

<IPython.core.display.Javascript object>